In [ ]:
sentences=['this is my favourite restaurent','I like to eat burger in this restaurent',
           'food is so spicy not eatable',
           'food is ok']
sentiment=['positive','positive','negative','neutral']

In [ ]:
import nltk
nltk.download('punkt')
from nltk.tokenize import word_tokenize
words=[]
for i in sentences:
  p=word_tokenize(i)
  print(p) 

['this', 'is', 'my', 'favourite', 'restaurent']
['I', 'like', 'to', 'eat', 'burger', 'in', 'this', 'restaurent']
['food', 'is', 'so', 'spicy', 'not', 'eatable']
['food', 'is', 'ok']


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [ ]:
!wget http://downloads.cs.stanford.edu/nlp/data/glove.6B.zip     # wordembided 

--2023-01-12 09:18:39--  http://downloads.cs.stanford.edu/nlp/data/glove.6B.zip
Resolving downloads.cs.stanford.edu (downloads.cs.stanford.edu)... 171.64.64.22
Connecting to downloads.cs.stanford.edu (downloads.cs.stanford.edu)|171.64.64.22|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 862182613 (822M) [application/zip]
Saving to: ‘glove.6B.zip.1’

glove.6B.zip.1      100%[===================>] 822.24M  5.09MB/s    in 2m 40s  

2023-01-12 09:21:18 (5.15 MB/s) - ‘glove.6B.zip.1’ saved [862182613/862182613]



In [ ]:
!unzip glove.6B.zip

Archive:  glove.6B.zip
replace glove.6B.50d.txt? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

In [ ]:
import pandas as pd
data=[['the restaurant is awsome',1],['the food is so oily and spicy',0],['food is ok',1]]
df=pd.DataFrame(data,columns=['review','sentiment'])
df

,review,sentiment
0,the restaurant is awsome,1
1,the food is so oily and spicy,0
2,food is ok,1


In [ ]:
def add_to_dict(filename,d):
  import numpy as np
  with open(filename,'r') as f:
    for line in f.readlines():
      l=line.split(' ')
      d[l[0]]=np.array(l[1:],dtype='float')

In [ ]:
dics={}
add_to_dict('glove.6B.50d.txt',dics)

In [ ]:
dics['the']

array([ 4.1800e-01,  2.4968e-01, -4.1242e-01,  1.2170e-01,  3.4527e-01,
       -4.4457e-02, -4.9688e-01, -1.7862e-01, -6.6023e-04, -6.5660e-01,
        2.7843e-01, -1.4767e-01, -5.5677e-01,  1.4658e-01, -9.5095e-03,
        1.1658e-02,  1.0204e-01, -1.2792e-01, -8.4430e-01, -1.2181e-01,
       -1.6801e-02, -3.3279e-01, -1.5520e-01, -2.3131e-01, -1.9181e-01,
       -1.8823e+00, -7.6746e-01,  9.9051e-02, -4.2125e-01, -1.9526e-01,
        4.0071e+00, -1.8594e-01, -5.2287e-01, -3.1681e-01,  5.9213e-04,
        7.4449e-03,  1.7778e-01, -1.5897e-01,  1.2041e-02, -5.4223e-02,
       -2.9871e-01, -1.5749e-01, -3.4758e-01, -4.5637e-02, -4.4251e-01,
        1.8785e-01,  2.7849e-03, -1.8411e-01, -1.1514e-01, -7.8581e-01])

In [ ]:
import nltk
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('omw-1.4')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


True

In [ ]:
from nltk.stem import WordNetLemmatizer   # stem convert the base form of the word
l=WordNetLemmatizer()
l.lemmatize('feet')
nltk.word_tokenize('hello how are you')    # word_tokenize break the sentence into word


['hello', 'how', 'are', 'you']

In [ ]:
def message_to_tokenlist(s):   # its convert the list of token
   tokens=nltk.word_tokenize(s)
   lower_case=[t.lower() for t in tokens]
   lemm=[l.lemmatize(t) for t in lower_case]
   usefultokens=[t for t in lemm if t in dics]
   return usefultokens

In [ ]:
message_to_tokenlist('I Am so happy at sistec')

['i', 'am', 'so', 'happy', 'at']

In [ ]:
def message_to_vector(s,dics):     # conert to message into vector
  usefultokens=message_to_tokenlist(s)
  vector=[]
  for token in usefultokens:
    if token not in dics:
      continue
    q=dics[token]
    vector.append(q)
  return np.array(vector,dtype='float')

In [ ]:
import numpy as np
a=message_to_vector('hello how are you',dics)
a.shape

(4, 50)

In [ ]:
# all sentences
def df_to_x_y(df,dics):
  y=df['sentiment'].to_numpy().astype('int')
  x=[]
  for message in df['review']:
    message_in_matrix=message_to_vector(message,dics)
    x.append(message_in_matrix)
  return x,y

In [ ]:
 x,y=df_to_x_y(df,dics)

In [ ]:
from copy import deepcopy
def pad_x(x,length):
  x_copy=[]
  for i in x:
   difference=length-i.shape[0]
   pad=np.zeros(shape=(difference,50))
   x_copy.append(np.concatenate([i,pad]))
  return np.array(x_copy)

In [ ]:
xarr=pad_x(x,7)
xarr.shape

(3, 7, 50)

In [ ]:
from keras.models import Sequential
model=Sequential()
from keras.layers import Dense,LSTM,Dropout,Input
model.add(Input(shape=(7,50)))
model.add(LSTM(64,activation='relu',return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(64,activation='relu',return_sequences=False))
model.add(Dropout(0.2))
model.add(Dense(1,activation='sigmoid'))
model.compile(loss='binary_crossentropy',metrics=['accuracy'],optimizer='adam')
model.fit(xarr,y,epochs=10)

Epoch 1/10
1/1 [==============================] - 4s 4s/step - loss: 0.7030 - accuracy: 0.3333
Epoch 2/10
1/1 [==============================] - 0s 19ms/step - loss: 0.6964 - accuracy: 0.3333
Epoch 3/10
1/1 [==============================] - 0s 23ms/step - loss: 0.6751 - accuracy: 0.3333
Epoch 4/10
1/1 [==============================] - 0s 16ms/step - loss: 0.6919 - accuracy: 0.6667
Epoch 5/10
1/1 [==============================] - 0s 15ms/step - loss: 0.6505 - accuracy: 1.0000
Epoch 6/10
1/1 [==============================] - 0s 12ms/step - loss: 0.6518 - accuracy: 1.0000
Epoch 7/10
1/1 [==============================] - 0s 16ms/step - loss: 0.6506 - accuracy: 1.0000
Epoch 8/10
1/1 [==============================] - 0s 15ms/step - loss: 0.6686 - accuracy: 1.0000
Epoch 9/10
1/1 [==============================] - 0s 15ms/step - loss: 0.6453 - accuracy: 1.0000
Epoch 10/10
1/1 [==============================] - 0s 12ms/step - loss: 0.6306 - accuracy: 1.0000


In [ ]:
o=model.predict(xarr)
#o
o>0.5

1/1 [==============================] - 1s 658ms/step


array([[ True],
       [False],
       [ True]])